In [3]:
from database import *

mdb = DatasetDB()
print(mdb.get_all_id())
print()
id = mdb.get_all_id()[0]
print(id)
print(mdb.get_config(id))

['ai_hub_korean_det', 'ai_hub_korean_rec', 'ICDAR2015', 'sangmu_MH_std', 'test1', 'test2']

ai_hub_korean_det
{'task': ['STD'], 'labelfiles': ['label1.txt'], 'options': [], 'id': 'ai_hub_korean_det'}


#### 레이블 셋 생성

In [4]:
from database import *
datadb = DatasetDB()
labeldb = LabelsetDB()
datadb.get_all_id()

labeldb.make("sangmu_std", ["sangmu_std_MH", "sangmu_std_DI", "sangmu_std_DH"], split_ratio=[8, 1, 1], random_seed=100)
labeldb.make("sangmu_str", ["sangmu_str_MH", "sangmu_str_DI", "sangmu_str_DH"], split_ratio=[8, 1, 1], random_seed=100)
labeldb.make("outsourcing_std", ["external_service1_std", "external_service1(exception)_std"], [8, 1, 1], random_seed=100)
labeldb.make("outsourcing_str", ["external_service1_str", "external_service1(exception)_str"], [8, 1, 1], random_seed=100)

labeldb.make("sangmu_and_outsourcing_std", ["external_service1_std", "external_service1(exception)_std"], [8, 2, 0], individual_split_ratio={
    "sangmu_std_MH":[0, 0, 1],
    "sangmu_std_DI":[0, 0, 1],
    "sangmu_std_DH":[0, 0, 1]
    }, random_seed=100)

labeldb.make("sangmu_and_outsourcing_str", ["external_service1_str", "external_service1(exception)_str"], [8, 2, 0], individual_split_ratio={
    "sangmu_str_MH":[0, 0, 1],
    "sangmu_str_DI":[0, 0, 1],
    "sangmu_str_DH":[0, 0, 1]
    }, random_seed=100)

images/1.png	[{"transcription": "IBK WM 센터 광주", "points": [[535, 27], [2683, 27], [2683, 343], [535, 343]], "difficult": false}, {"transcription": "IBK 기업은행", "points": [[2777, 63], [3343, 63], [3343, 163], [2777, 163]], "difficult": false}, {"transcription": "IBK 투자증권", "points": [[2797, 171], [3362, 171], [3362, 274], [2797, 274]], "difficult": false}]
images/1.png	[{"transcription": "NH농협은행", "points": [[226, 67], [808, 67], [808, 172], [226, 172]], "difficult": false}, {"transcription": "365 Auto Bank", "points": [[1010, 132], [2332, 132], [2332, 376], [1010, 376]], "difficult": false}]
images/1.png	[{"transcription": "TIMESTOWER", "points": [[16, 6], [153, 9], [147, 39], [475, 52], [509, 13], [636, 18], [642, 41], [1016, 59], [1051, 67], [1071, 179], [25, 159], [12, 40]], "difficult": false}]
2. save label files
3. make infer label files
E:\workspace\paddleocr\datasets\sangmu_str_MH\1\1.png	IBK WM 센터 광주
E:\workspace\paddleocr\datasets\sangmu_str_DI\1\1.png	NH농협은행
E:\workspace\padd

#### k_fold 레이블 셋 생성

In [5]:
# from database import *
# labeldb = LabelsetDB()
# labeldb.make_k_fold(["test2"], "test_k_fold", k=5)
# labeldb.get_config("test_k_fold_5_1")

##### Model

In [1]:
from database import *
modeldb = ModelDB()
import os
# modeldb.get_config("en_PP-OCRv3_det")
modeldb.get_all_id()
modeldb.get_config("MobileNetV3_large_x0_5")
# for id in modeldb.get_all_id():
#     print(id)
#     modeldb.make_inference_model(id)

{'id': 'MobileNetV3_large_x0_5',
 'inference_model': './models/MobileNetV3_large_x0_5/inference_model/inference.pdmodel',
 'inference_model_dir': './models/MobileNetV3_large_x0_5/inference_model',
 'inference_model_weight': './models/MobileNetV3_large_x0_5/inference_model/inference.pdiparams',
 'pretrained_model_dir': './models/MobileNetV3_large_x0_5/pretrained_model',
 'pretrained_model_weight': './models/MobileNetV3_large_x0_5/pretrained_model/pretrained.pdparams',
 'train_config': './models/MobileNetV3_large_x0_5/pretrained_model/config.yml'}

##### Work

In [9]:
from database import *
workdb = WorkDB()
modeldb = ModelDB()
labelsetdb = LabelsetDB()
modeldb.get_all_id()
workdb.make("det_PPOCR_sangmu", ["sangmu_std"], "ml_PP-OCRv3_det")
workdb.make("det_PPOCR_oursourcing", ["outsourcing_std"], "ml_PP-OCRv3_det")
workdb.make("det_PPOCR_sangmu_and_oursourcing", ["sangmu_and_outsourcing_std"], "ml_PP-OCRv3_det")

In [10]:
workdb.train("det_PPOCR_sangmu", 10) 
workdb.train("det_PPOCR_oursourcing", 10) 
workdb.train("det_PPOCR_sangmu_and_oursourcing", 10)

In [7]:
from database import *
import project
from pathlib import Path
# config = "configs\det\det_mv3_db.yml"
# labelsets = ["test", "test_k_fold_5_1"]
# model = "MobileNetV3_large_x0_5"
# name = "test"
# WorkDB().make(name, labelsets, model, config)

WorkDB().train("test", 30)


In [ ]:

from database import *
from pathlib import Path

def full_infer(image_dir, det_model_dir, rec_model_dir, save_dir):
    code = 'code/PaddleOCR/tools/infer/predict_system.py'
    det_algorithm="DB"

    # rec_model_dir = "E:/workspace/paddleocr/outputs/rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune/trained_model/inference_model"
    rec_char_dict_path = "E:/workspace/paddleocr/code/PaddleOCR/ppocr/utils/dict/korean_dict.txt"
    vis_font_path = "C:\Windows\Fonts\malgun.ttf"

    command = f"""python {code} --image_dir={image_dir} \
    --det_model_dir={det_model_dir} \
    --det_algorithm={det_algorithm} \
    --rec_model_dir={rec_model_dir} \
    --rec_image_shape="3, 32, 100" \
    --rec_char_dict_path={rec_char_dict_path} \
    --vis_font_path={vis_font_path} \
    --draw_img_save_dir={save_dir} \
    --drop_score=0.3 \
    --det_db_box_thresh=0.4"""
    
    path = Path(project.PROJECT_ROOT)/"inference/run.sh"
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a") as f:
        f.write(command+"\n")
    
modeldb = ModelDB()
workdb = WorkDB()

# det_model_dir = modeldb.get_config("MobileNetV3_large_x0_5")["inference_model_dir"] 
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/Mobile_PaddleOCR_pretrained"
# full_infer(det_model_dir, rec_model_dir, save_dir)

# image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
# det_model_dir = modeldb.get_config("ml_PP-OCRv3_det")["inference_model_dir"]
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PaddleOCR_PaddleOCR_pretrained_sangmu"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_sangmu"
full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

image_dir = "./datasets/ai_hub_korean_det_partial_test/images"
det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_aihub"
full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)



In [4]:
from pathlib import Path

Path("./datasets/ai_hub_korean_image_det/train2/source/간판/돌출간판/돌출간판1/간판_돌출간판_001449.jpg").exists()

False

In [10]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt") as f:
    lines = [line.rstrip("\n") for line in f.readlines()]

a, b, c = 0, 0, 0
for line in lines:
    if "가로형간판" in line:
        a+= 1
    elif "세로형간판" in line:
        b+= 1
    elif "돌출간판" in line:
        c+= 1
print(a, b, c)

80242 25621 55803


In [11]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label3.txt", encoding="cp949") as f:
    with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt", "w", encoding="utf-8") as f2:
        for line in f.readlines():
            f2.write(line)

In [4]:
from dataset.aihub.hangul_real_image_dataset import *
dataset = HangulRealImageDataset()


*******Checked Dir List*******
datasets\ai_hub_korean_det\train1\source\간판\세로형간판
datasets\ai_hub_korean_det\val1\source\간판\세로형간판
datasets\ai_hub_korean_det\train2\source\간판\가로형간판\가로형간판1
datasets\ai_hub_korean_det\train2\source\간판\돌출간판\돌출간판1
datasets\ai_hub_korean_det\train2\source\간판\세로형간판
datasets\ai_hub_korean_det\val2\source\간판\세로형간판
******************************


In [6]:
dataset = HangulRealImage_BoxDetectionDataset(dataset)

In [29]:
x, y = dataset[0]
dataset.show_xy(x, y)
y

[{'bbox': [[762, 306], [849, 306], [849, 518], [762, 518]], 'label': '소망'},
 {'bbox': [[754, 547], [837, 547], [837, 900], [754, 900]], 'label': '미용실'},
 {'bbox': [[730, 1053], [845, 1053], [845, 1147], [730, 1147]],
  'label': 'xxx'},
 {'bbox': [[871, 1078], [1047, 1078], [1047, 1139], [871, 1139]],
  'label': 'xxx'},
 {'bbox': [[1177, 732], [1479, 732], [1479, 794], [1177, 794]],
  'label': 'xxx'},
 {'bbox': [[1429, 707], [1502, 707], [1502, 748], [1429, 748]],
  'label': 'xxx'},
 {'bbox': [[1342, 1075], [1428, 1075], [1428, 1160], [1342, 1160]],
  'label': 'xxx'},
 {'bbox': [[184, 345], [361, 345], [361, 611], [184, 611]], 'label': 'xxx'},
 {'bbox': [[121, 291], [184, 291], [184, 342], [121, 342]], 'label': 'xxx'},
 {'bbox': [[365, 194], [460, 194], [460, 252], [365, 252]], 'label': 'xxx'},
 {'bbox': [[391, 536], [443, 536], [443, 587], [391, 587]], 'label': 'xxx'},
 {'bbox': [[355, 610], [517, 610], [517, 684], [355, 684]], 'label': 'xxx'},
 {'bbox': [[172, 748], [240, 748], [240, 

In [53]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"epoch":[], "step":[], "acc":[], "loss":[], "precision":[], "recall":[]})

df = df._append({"epoch":1, "step":2, "acc":3, "loss":4, "precision":5, "recall":1}, ignore_index=True)
df.to_csv("test.csv")
df

,epoch,step,acc,loss,precision,recall
0,1.0,2.0,3.0,4.0,5.0,1.0


In [55]:
df = pd.read_csv("test.csv", index_col=0)
df

,epoch,step,acc,loss,precision,recall
0,1.0,2.0,3.0,4.0,5.0,1.0


In [60]:
from database import *

WorkDB().get_all_id()
WorkDB().get_config("det_MobileNet_sangmu")

{'id': 'det_MobileNet_sangmu',
 'labelsets': ['sangmu'],
 'model': 'MobileNetV3_large_x0_5',
 'pretrained': True,
 'train_config': './works/det_MobileNet_sangmu/train_config.yml',
 'result_path': './works/det_MobileNet_sangmu/result.csv'}

In [6]:
from database import *

WorkDB().get_all_id()
# WorkDB().infer_det("det_MobileNet_sangmu")
# WorkDB().get_best_model_weight("det_MobileNet_sangmu")
# WorkDB().infer_det("det_MobileNet_sangmu", "test", "best")
WorkDB().infer_det("det_PPOCR_sangmu", "best", "test")

In [ ]:
path = "F:/5.SCI Lab/1.과제 & 연구/[2023] 불법옥외간판(외주)/데이터셋/레이블링 결과(최종)_0126/그외 간판"

wi

[60, 140, 200]

In [7]:
import yaml

with open("E:/workspace/paddleocr/labelsets/test1/config.yml") as f:
    x = yaml.load(f, Loader=yaml.FullLoader)
x

{'dataset_dir': './datasets',
 'datasets': ['sangmu_std_MH', 'sangmu_std_DI', 'sangmu_std_DH'],
 'infer': {'eval': ['eval_infer.txt'],
  'test': ['test_infer.txt'],
  'train': ['train_infer.txt']},
 'label': {'eval': ['eval_label.txt'],
  'test': ['test_label.txt'],
  'train': ['train_label.txt']},
 'seed': 100,
 'split': {'sangmu_std_DH': [8, 1, 1],
  'sangmu_std_DI': [8, 1, 1],
  'sangmu_std_MH': [8, 1, 1]}}

In [3]:
x = {1:1, 2:2}
y = {1:10, 2:20}

x.update(y)
x

{1: 10, 2: 20}